In [1]:
import os
os.chdir("/Users/kibeomkim/Desktop/")
def read_data(filename):
    with open(filename, 'r', encoding="UTF-8") as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]                                       # 파일의 헤더(컬럼명) 제외
    return data

data = read_data('ratings_morphed.txt')    # 1은 긍정, 0은 부정

# 훈련데이터와 테스트데이터로 분리
data_text = [line[1] for line in data]             # 훈련데이터 본문
data_senti = [line[2] for line in data]           # 훈련데이터 긍부정 부분

from sklearn.model_selection import train_test_split
train_data_text, test_data_text, train_data_senti, test_data_senti = train_test_split(data_text, data_senti, stratify=data_senti)

# Counter 클래스를 이용해 각 분류가 같은 비율로 들어갔는지 확인해 본다
from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
print('train_data_senti_freq:', train_data_senti_freq)

test_data_senti_freq = Counter(test_data_senti)
print('test_data_senti_freq:', test_data_senti_freq)

train_data_senti_freq: Counter({'1': 74097, '0': 74058})
test_data_senti_freq: Counter({'1': 24700, '0': 24686})


In [ ]:
# 훈련용 데이터셋을 가지고 문장 별로 문자열로 구성된 임의의 벡터를 생성한다. 
# 그 후 벡터 각 원소를 문장 내 빈도수, 그러니까 숫자로 바꾸어준다. 
# 벡터들을 쌓아서 행렬을 만든다. 이 행렬은 훈련용 데이터셋이 형태만 변환된 것이다. 
# 이 행렬을 앞으로 훈련용 데이터셋으로 쓸 것이다. 
# CountVectorizer 가 데이터셋을 행렬 형태로 변환해주는 클래스다. 
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5).fit(train_data_text)
X_train = vect.transform(train_data_text) # 훈련용 데이터셋을 행렬 형태로 변환
print("X_train:\n", repr(X_train))
feature_names = vect.get_feature_names()
print("특성 개수:", len(feature_names))
print("처음 20개 특성:\m", feature_names[:20])
print("3000~5000까지의 특성:\n", feature_names[3000:5000])

In [3]:
import pandas as pd # 데이터프레임 생성 위해 판다스 호출
from sklearn.model_selection import cross_val_score # 사이킷런 교차검증 모델 성능 점수 계산 위해 호출
from sklearn.linear_model import LogisticRegression # 사이킷런 선형모형 중 로지스틱 회귀모델 (머신러닝방식) 호출
y_train = pd.Series(train_data_senti) # 훈련용 정답 데이터를 판다스 시리즈 형식 데이터로 변환
scores = cross_val_score(LogisticRegression(solver="liblinear"), X_train, y_train, cv=5) # 로지스틱 회귀모형을 X_train & y_train 데이터 사용해서 5-fold 교차검증 
print('교차 검증 점수:', scores)# 5번 교차검증 한 다음 계산해낸 성능점수 5개 
print('교차 검증 점수 평균:', scores.mean()) # 5개의 평균이 곧 모형의 성능점수다. 

교차 검증 점수: [0.80803888 0.80830887 0.80689143 0.80452904 0.80844386]
교차 검증 점수 평균: 0.8072424150383044


In [4]:
from sklearn.model_selection import GridSearchCV # 그리드서치 최적화 알고리즘 (성능을 최대로 만드는 최적 입력 파라미터 찾기)
param_grid = {'C': [0.01, 0.1, 1, 3, 5]} # 후보 파라미터 
grid = GridSearchCV(LogisticRegression(solver="liblinear"), param_grid, cv=5) # 그리드 서치 방법으로 로지스틱 분류모형 최적화 
grid.fit(X_train, y_train) 
print("최고 교차 검증 점수:", round(grid.best_score_, 3)) # 최적 파라미터 값에서 분류모형 최대 성능값
print("최적의 매개변수:", grid.best_params_) # 분류모형 최대 성능값 나오게 하는 최적 파라미터 입력값 : 1

최고 교차 검증 점수: 0.807
최적의 매개변수: {'C': 1}


In [5]:
X_test = vect.transform(test_data_text) # 검정용 데이터를 벡터로 변환 
y_test = pd.Series(test_data_senti) # 검정용 정답데이터
print("테스트 데이터 점수:", grid.score(X_test, y_test)) # 검정용 데이터를 모형에 넣었을 때 모형 성능 점수 

테스트 데이터 점수: 0.8087919653343053


In [6]:
import rhinoMorph
rn = rhinoMorph.startRhino()

new_input = '사랑하자'
# 입력 데이터 형태소 분석하기 
inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
morphed_input = ' '.join(morphed_input)                     # 한 개의 문자열로 만들기
inputdata.append(morphed_input)                               # 분석 결과를 리스트로 만들기
X_input = vect.transform(inputdata)
result = grid.predict(X_input) # 0은 부정,1은 긍정

if result[0] == '0' : 
    print("부정적인 글입니다")
else:
    print("긍정적인 글입니다")


filepath:  /Users/kibeomkim/opt/anaconda3/lib/python3.8/site-packages
classpath:  /Users/kibeomkim/opt/anaconda3/lib/python3.8/site-packages/rhinoMorph/lib/rhino.jar
RHINO started!
긍정적인 글입니다


# 학습된 모델에 내 데이터 적용하기 

In [ ]:
df = pd.read_csv('/Users/kibeomkim/Desktop/유튜브 광고 데이터셋/50퍼센트 이상.csv') 
nalchi = df[1:1826].iloc[:, 1:6] 
nalchi.columns = ['주소', '작성자', '작성날짜', '내용', '공감수']
cont = list(nalchi['내용'])
cont

In [8]:
import rhinoMorph
rn = rhinoMorph.startRhino()

positive_list = []
negative_list = []

for text in cont : 
    new_input = text
    # 입력 데이터 형태소 분석하기 
    inputdata = []
    morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
    morphed_input = ' '.join(morphed_input)                     # 한 개의 문자열로 만들기
    inputdata.append(morphed_input)                               # 분석 결과를 리스트로 만들기
    X_input = vect.transform(inputdata)
    result = grid.predict(X_input) # 0은 부정,1은 긍정

    if result[0] == '0' : 
        negative_list.append(result[0])
        print("부정적인 글입니다")
    else:
        positive_list.append(result[0])
        print("긍정적인 글입니다")

filepath:  /Users/kibeomkim/opt/anaconda3/lib/python3.8/site-packages
classpath:  /Users/kibeomkim/opt/anaconda3/lib/python3.8/site-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
부정적인 글입니다
부정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
부정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
부정적인 글입니다
부정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
부정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
긍정적인 글입니다
부정적인 글입니다
부정적인 

In [9]:
new_input = cont[17]
# 입력 데이터 형태소 분석하기 
inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
morphed_input = ' '.join(morphed_input)                     # 한 개의 문자열로 만들기
inputdata.append(morphed_input)                               # 분석 결과를 리스트로 만들기
X_input = vect.transform(inputdata)
result = grid.predict(X_input) # 0은 부정,1은 긍정

if result[0] == '0' : 
    print("부정적인 글입니다")
else:
    print("긍정적인 글입니다")

부정적인 글입니다


In [10]:
cont[17]

'난진짜 랩이랑 판소리랑 섞일줄 몰랐어,, 섞였는데 그게 미친듯이 좋을 줄 몰랐어,,,그 미친듯이 좋은게 휴대폰광고일준 몰랐어,,,,'

In [11]:
print(f'긍정 : {len(positive_list)}')
print(f'부정 : {len(negative_list)}')

긍정 : 720
부정 : 1105


In [ ]:
sns.barplot(x=['긍정','부정'], y=[len(positive_list), len(negative_list)])
plt.text(0,700, f'{round(len(positive_list)/len(cont),2)}')
plt.text(1,1150, f'{round(len(negative_list)/len(cont),2)}')

In [ ]:
plt.figure(figsize=(5,5))
plt.pie([0.38, 0.62], autopct = '%1.2f', labels=['긍정', '부정'], )
plt.axis('equal')
plt.title('감성분석 결과')

## 전체 데이터셋에 학습된 모델 적용

In [95]:
df = pd.read_csv('/Users/kibeomkim/Desktop/유튜브 광고 데이터셋/50퍼센트 이상.csv') 
df.columns = ['삭제', '주소', '작성자', '작성일자', '내용', '공감수', '삭제2']
df = df.iloc[:, 1:6]

In [ ]:
nalchi = df[1:1826]
see_capture = df[1827:2041]
spen = df[2042:2111]
togo = df[2112:2231]
ultra = df[2232:2263]
memory = df[2264:2294]
unboxing = df[2295:3037]
jp_ep3 = df[3038:3347]
nalchi_unboxing = df[3348:3547]
panthom = df[3548:3571]
epic_in_everyway = df[3572:3579]
bang = df[3580:3597]
nobcut = df[3598:3695]
imperfect = df[3696:3724]

name_list = [nalchi, see_capture, spen, togo, ultra, memory, unboxing, jp_ep3, nalchi_unboxing, panthom, epic_in_everyway, bang, nobcut, imperfect]
names = ['nalchi', 'see_capture', 'spen', 'togo', 'ultra', 'memory', 'unboxing', 'jp_ep3', 'nalchi_unboxing', 'panthom', 'epic_in_everyway', 'bang', 'nobcut', 'imperfect']



# 전처리
name_list2 = []
for name in name_list : 
    name.drop_duplicates(subset=['내용'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
    name = name.dropna(how = 'any') # Null 값이 존재하는 행 제거
    name['내용'] = name['내용'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
    name['내용'] = name['내용'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
    name['내용'].replace('', np.nan, inplace=True)
    name = name.dropna(how = 'any') # Null 값이 존재하는 행 제거

    name['내용'] = name['내용'].str.replace('광고', '')
    name['내용'] = name['내용'].str.replace('찍었다', '촬영했다')
    name['내용'] = name['내용'].str.replace('찍었', '촬영했')

    name_list2.append(name)








p2 = sns.color_palette('colorblind', 10)
name_dict = dict()
no = 1
for name in name_list2 : 
    positive_list = []
    negative_list = []
    cont = list(name['내용'])
    
    for text in cont : 
        try : 
            new_input = text
            # 입력 데이터 형태소 분석하기 
            inputdata = []
            morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
            morphed_input = ' '.join(morphed_input)                     # 한 개의 문자열로 만들기
            inputdata.append(morphed_input)                               # 분석 결과를 리스트로 만들기
            X_input = vect.transform(inputdata)
            result = grid.predict(X_input) # 0은 부정,1은 긍정

            if result[0] == '0' : 
                negative_list.append(result[0])
                #print("부정적인 글입니다")
            else:
                positive_list.append(result[0])
                #print("긍정적인 글입니다")
        except : 
            #print('댓글 이상')
            continue
    
    positive_num = len(positive_list)
    negative_num = len(negative_list)

    name_dict[no] = positive_num, negative_num
    
    sns.barplot(x=['긍정','부정'], y=[positive_num, negative_num], palette=p2)
    #plt.text(0,700, f'{round(positive_num/len(cont),2)}')
    #plt.text(1,1150, f'{round(negative_num/len(cont),2)}')
    plt.title(f'{names[no-1]}')

    f_dir = f'/Users/kibeomkim/Desktop/감성분석_결과_50% 이상'
    os.makedirs(f_dir+'/'+names[no-1])
    os.chdir(f_dir+'/'+names[no-1])
    plt.savefig(f'{no-1}.jpg', dpi=300)

    plt.pause(0.5)
    plt.close()

    #plt.figure(figsize=(5,5))
    plt.title(f'{names[no-1]}')
    plt.pie([round(positive_num/len(cont),2), round(negative_num/len(cont),2)], autopct = '%1.2f', labels=['긍정', '부정'], colors=['cyan', 'lightcoral'])
    #plt.axis('equal')

    plt.savefig(f'{no}.jpg', dpi=300)
    plt.pause(0.5)
    plt.close()

    no += 1


# 50% 이상인 그룹 - 감성분석 결과 정리 

In [97]:
pos_neg = pd.DataFrame(name_dict)
pos_neg.columns = ['nalchi', 'see_capture', 'spen', 'togo', 'ultra', 'memory', 'unboxing', 'jp_ep3', 'nalchi_unboxing', 'panthom', 'epic_in_everyway', 'bang', 'nobcut', 'imperfect']
pos_neg.index = ['긍정','부정']
pos_neg

,nalchi,see_capture,spen,togo,ultra,memory,unboxing,jp_ep3,nalchi_unboxing,panthom,epic_in_everyway,bang,nobcut,imperfect
긍정,895,90,36,36,13,15,422,192,98,14,3,8,48,18
부정,887,119,29,79,16,15,307,112,99,9,3,7,45,10


# 50% 이하인 그룹 - 감성분석

In [98]:
df1 = pd.read_csv('/Users/kibeomkim/Desktop/유튜브 광고 데이터셋/50퍼센트 이하.csv') 
df1.columns = ['삭제', '주소', '작성자', '작성일자', '내용', '공감수', '삭제']
df1 = df1.iloc[1:, 1:6]

In [99]:
df1

,주소,작성자,작성일자,내용,공감수
1,"[갤럭시 s21] 한몸에 받다. 세상의 시선 / 조회수 : 2,449,304 / 광...",NaN,NaN,NaN,NaN
2,https://www.youtube.com/watch?v=bC9TksWbq-I,스트민,10 months ago,와 브금이랑 진짜 모든게 개간지나네,266
3,https://www.youtube.com/watch?v=bC9TksWbq-I,김보정,10 months ago,애플과 또다른 세련된 감성..,167
4,https://www.youtube.com/watch?v=bC9TksWbq-I,Daehwan Kim,10 months ago,깔끔하고 세련된 광고. 재드래곤 믿고 삼전갑니다.,166
5,https://www.youtube.com/watch?v=bC9TksWbq-I,김모죠,10 months ago,광고 진짜 많이 예뻐졌다,375
...,...,...,...,...,...
2310,https://www.youtube.com/watch?v=PbZxsRIOBBE&li...,주박 JBak,9 months ago,김 ㄹ 님 ㄷㄷ김 리을,2
2311,https://www.youtube.com/watch?v=PbZxsRIOBBE&li...,김철곤,17 hours ago,카메라부문 클리얼블랙으로. 버즈들도 클리어 한정판을주라디자인이라고뻥치는것보다 차라리...,0
2312,https://www.youtube.com/watch?v=PbZxsRIOBBE&li...,킹갓엠페럴제네럴황제충무공드래곤,8 months ago (edited),광고 브금 이름...제발,0
2313,https://www.youtube.com/watch?v=PbZxsRIOBBE&li...,ks,9 months ago,Z플립3 빨리 내주세여,0


In [ ]:
acceptview = df1[1:269]
erase = df1[271:301]
nowimreleased = df1[303:329]
yupeng_full = df1[331:787]
yupeng_ep2 = df1[789:1011]
yupeng_ep1 = df1[1013:1495]
zipcok_party = df1[1497:1597]
mydream = df1[1599:1630]
tongdong = df1[1632:1645]
bts_feature = df1[1647:1698]
bts_epic_8k = df1[1701:1857]
phonecasting = df1[1860:2085]
bts_night = df1[2087:2159]
son_commentary = df1[2161:2173]
son = df1[2175:2282]
designer = df1[2283:2314]


name_list = [acceptview, erase, nowimreleased, yupeng_full, yupeng_ep2, yupeng_ep1, zipcok_party, mydream, tongdong, bts_feature, bts_epic_8k, phonecasting, bts_night, son_commentary, son, designer]
names = ['acceptview', 'erase', 'nowimreleased', 'yupeng_full', 'yupeng_ep2', 'yupeng_ep1', 'zipcok_party', 'mydream', 'tongdong', 'bts_feature', 'bts_epic_8k', 'phonecasting', 'bts_night', 'son_commentary', 'son', 'designer']

# 전처리

name_list2 = []
for name in name_list : 
    name.drop_duplicates(subset=['내용'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
    name = name.dropna(how = 'any') # Null 값이 존재하는 행 제거
    name['내용'] = name['내용'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
    name['내용'] = name['내용'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
    name['내용'].replace('', np.nan, inplace=True)
    name = name.dropna(how = 'any') # Null 값이 존재하는 행 제거

    name['내용'] = name['내용'].str.replace('광고', '')
    name['내용'] = name['내용'].str.replace('찍었다', '촬영했다')
    name['내용'] = name['내용'].str.replace('찍었', '촬영했')
    name_list2.append(name)




p2 = sns.color_palette('colorblind', 10)
name_dict = dict()
no = 1
for name in name_list2 : 
    positive_list = []
    negative_list = []
    cont = list(name['내용'])
    
    for text in cont : 
        try : 
            new_input = text
            # 입력 데이터 형태소 분석하기 
            inputdata = []
            morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
            morphed_input = ' '.join(morphed_input)                     # 한 개의 문자열로 만들기
            inputdata.append(morphed_input)                               # 분석 결과를 리스트로 만들기
            X_input = vect.transform(inputdata)
            result = grid.predict(X_input) # 0은 부정,1은 긍정

            if result[0] == '0' : 
                negative_list.append(result[0])
                #print("부정적인 글입니다")
            else:
                positive_list.append(result[0])
                #print("긍정적인 글입니다")
        except : 
            #print('댓글 이상')
            continue
    
    positive_num = len(positive_list)
    negative_num = len(negative_list)

    name_dict[no] = positive_num, negative_num
    
    sns.barplot(x=['긍정','부정'], y=[positive_num, negative_num], palette=p2)
    #plt.text(0,700, f'{round(positive_num/len(cont),2)}')
    #plt.text(1,1150, f'{round(negative_num/len(cont),2)}')
    plt.title(f'{names[no-1]}')

    f_dir = f'/Users/kibeomkim/Desktop/감성분석결과_50% 이하'
    os.makedirs(f_dir+'/'+names[no-1])
    os.chdir(f_dir+'/'+names[no-1])
    plt.savefig(f'{no-1}.jpg', dpi=300)

    plt.pause(0.5)
    plt.close()

    #plt.figure(figsize=(5,5))
    plt.title(f'{names[no-1]}')
    plt.pie([round(positive_num/len(cont),2), round(negative_num/len(cont),2)], autopct = '%1.2f', labels=['긍정', '부정'], colors=['cyan', 'lightcoral'])
    #plt.axis('equal')

    plt.savefig(f'{no}.jpg', dpi=300)
    plt.pause(0.5)
    plt.close()

    no += 1


# 50% 이하인 그룹_감성분석 결과 정리 

In [101]:
pos_neg = pd.DataFrame(name_dict)
pos_neg.columns = ['acceptview', 'erase', 'nowimreleased', 'yupeng_full', 'yupeng_ep2', 'yupeng_ep1', 'zipcok_party', 'mydream', 'tongdong', 'bts_feature', 'bts_epic_8k', 'phonecasting', 'bts_night', 'son_commentary', 'son', 'designer']
pos_neg.index = ['긍정','부정']
pos_neg

,acceptview,erase,nowimreleased,yupeng_full,yupeng_ep2,yupeng_ep1,zipcok_party,mydream,tongdong,bts_feature,bts_epic_8k,phonecasting,bts_night,son_commentary,son,designer
긍정,134,16,9,252,117,258,40,17,5,23,87,122,32,6,61,22
부정,120,14,15,187,103,218,45,10,6,21,57,101,27,6,42,9
